In [30]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
import itertools
import copy

In [31]:
# reading the given file.
newdf = pd.read_csv("indemed new entry sheet 4000.csv")

In [3]:
df = pd.DataFrame(data = newdf['SHORT DESCRIPTION'], columns=['SHORT DESCRIPTION'])
df['FullSentenceRatio'] = 0

In [4]:
# find the adjacent ratio of all SHORT DESCRIPTION.
temp = df['SHORT DESCRIPTION'][0]
for index,rows in df.iterrows():
    df.loc[index,'FullSentenceRatio'] = fuzz.ratio(rows[0].lower(), temp.lower())
    temp = rows[0]

In [5]:
#Creating final list(contains batches) with ratio >= 70.
final=[]
semi_final=[]
for index, rows in df.iterrows():
    if index==0:
        semi_final.append(rows['SHORT DESCRIPTION'])
        continue
    if rows['FullSentenceRatio']>=70:
        semi_final.append(rows['SHORT DESCRIPTION'])
    else:
        final.append(semi_final)
        semi_final=[]
        semi_final.append(rows['SHORT DESCRIPTION'])

In [6]:
# Separating Single_line item from final list.
single_line=[]
for i in final:
    if len(i)==1:
        single_line.append(i)
count=0
for i in single_line:
    if i in final:
        final.remove(i)
        single_line[count]=i[0]
        count+=1

In [7]:
copy_final = copy.deepcopy(final)
copy_single_line = copy.deepcopy(single_line)

#### In next 5 cell we are extracting Parent Name of final list's Batch. 
#### matrix_list is created -> Batches with parent name.
#### batch_wo_pn -> Batches without parent name.

In [8]:
def remove_commas(batch):
    for i in range(len(batch)):
        batch[i] = batch[i].split(',')
        batch[i] = ''.join(batch[i])
        batch[i] = batch[i].replace('  ',' ')
    return batch

In [9]:
l2=[]
for i in final:
    batch = remove_commas(i)
    l2.append(batch)

In [10]:
def calculate_shortest_string(batch):
    max_num = 10000
    string = []
    for i in batch:
        if len(i) < max_num:
            max_num = len(i)
            string = i
    return string

In [11]:
matrix_list = []
batch_wo_pn = []

In [12]:
for i,tf in zip(l2,copy_final):
    parent_name = []
    sht_str = calculate_shortest_string(i)
    sht_str = sht_str.split()
    for j in range(len(sht_str)):
        flag=0
        for k in i:
            k = k.split()
            if sht_str[j].lower()==k[j].lower():
                flag=1
            else:
                flag=0
                break
        if flag==0:
            break
        if flag:
            parent_name.append(sht_str[j])       
    pn = ' '.join(parent_name)
    if pn == '':
        for si in tf:
            batch_wo_pn.append(si)        
    else:     
        matrix_list.append(str(pn))
        matrix_list +=  tf
        matrix_list.append(np.nan)

In [13]:
# Applying the same rule which is used above on df with threshold_ratio of 85.
bwpn_df = pd.DataFrame(batch_wo_pn, columns=['Batch w/o pn'])
bwpn_df['Ratio'] = 0

In [14]:
temp = bwpn_df['Batch w/o pn'][0]
for index,rows in bwpn_df.iterrows():
        bwpn_df.loc[index,'Ratio'] = fuzz.ratio(rows[0].lower(), temp.lower())
        temp = rows[0]

In [15]:
bwpn_final=[]
semi_final=[]
for index, rows in bwpn_df.iterrows():
    if index==0:
        semi_final.append(rows['Batch w/o pn'])
        #final.append(semi_final)
        continue
    if rows['Ratio']>=85:
        semi_final.append(rows['Batch w/o pn'])
    else:
        bwpn_final.append(semi_final)
        semi_final=[]
        semi_final.append(rows['Batch w/o pn'])

In [16]:
copy_bwpn_final = copy.deepcopy(bwpn_final)

In [17]:
def remove_commas(batch):
    for i in range(len(batch)):
        batch[i] = batch[i].split(',')
        batch[i] = ''.join(batch[i])
        batch[i] = batch[i].replace('  ',' ')
    return batch

In [18]:
l2=[]
for i in bwpn_final:
    batch = remove_commas(i)
    l2.append(batch)

In [19]:
def calculate_shortest_string(batch):
    max_num = 10000
    string = []
    for i in batch:
        if len(i) < max_num:
            max_num = len(i)
            string = i
    return string

In [20]:
new_batch_wo_pn = []

In [21]:
for i,tf in zip(l2,copy_bwpn_final):
    if len(i)!=1:
        parent_name = []
        sht_str = calculate_shortest_string(i)
        sht_str = sht_str.split()
        for j in range(len(sht_str)):
            flag=0
            for k in i:
                k = k.split()
                if sht_str[j].lower()==k[j].lower():
                    flag=1
                else:
                    flag=0
                    break
            if flag==0:
                break
            if flag:
                parent_name.append(sht_str[j])       
        pn = ' '.join(parent_name)
        if pn == '':
            for si in tf:
                new_batch_wo_pn.append(si)        
        else:     
            matrix_list.append(str(pn))
            matrix_list +=  tf
            matrix_list.append(np.nan)
    else:
        single_line.append(tf)

In [22]:
finaldf = pd.DataFrame(columns = newdf.columns)

In [23]:
def make_df(batch):
    global finaldf
    batch_to_series = pd.Series(batch)
    uniquebatch = batch_to_series.unique()
    uniquebatch = list(uniquebatch)
    for i in uniquebatch:
        tempdf = newdf[newdf['SHORT DESCRIPTION']==i]
        finaldf = finaldf.append(tempdf,ignore_index=True)

In [24]:
def insert_parent_row(row):
    global finaldf
    df_length = len(finaldf)
    finaldf.loc[df_length] = row

In [25]:
temp = []
for i in range(len(matrix_list)):
    if pd.isnull(matrix_list[i]):
        make_df(temp)
        temp = []
    elif i==0 or pd.isnull(matrix_list[i-1]):
        row = [np.nan]*30
        row[5] = matrix_list[i]
        insert_parent_row(row)
    else:
        temp.append(matrix_list[i])   

In [26]:
finaldf.to_csv('FinalDF.csv',index=False)

#### Finaldf has parent name extracted from the batched made from the indemed sheet.

In [27]:
single_line = pd.Series(single_line, name='Short Description')

In [28]:
new_batch_wo_pn = pd.Series(new_batch_wo_pn, name='Short Description')

In [29]:
single_line.to_csv('Single line item.csv', index=False)
new_batch_wo_pn.to_csv('Batches PN not extracted.csv', index=False)